In [35]:
# from utils import *
import random
import Graph as G
import numpy as np
import pandas as pd
import pickle
import networkx as nx


In [97]:
class VecDB():
    def __init__(self,R,L,alpha,K,datasetPath='dataset/DB1K.csv'):
        self.R=R
        self.L=L
        self.alpha=alpha
        self.k=K
        self.datasetPath=datasetPath
        self.DBGraph=self.Initialize_Random_Graph()
        self.medoid=self.get_medoid()
    #TODO: make another constructor
        
    def load_binary_data(self, binary_file_path):
    # Load the data from the binary file
        with open(binary_file_path, 'rb') as f:
            data = pickle.load(f)
        return data
    
    # Get vertices from DB and insert them into the graph
    def Initialize_Random_Graph(self):
        DBGraph=G.Graph()
        DBdata = self.load_binary_data(self.datasetPath)
        for row in DBdata.itertuples(index=True, name='Pandas'):
            dataKey = int(row[1])
            dataValue= np.array(row[2:],dtype=float)
            DBGraph.add_vertex(G.Vertex(dataKey, dataValue))
        
        size= len(DBGraph.verticies)
        if(size==0 or size==1):
            return
        
        #Building Edges
        for vertex in DBGraph:
            for i in range(self.R):
                neighbor= DBGraph.get_vertex(int(random.random()*size))
                while(neighbor==vertex):
                    neighbor= DBGraph.get_vertex(int(random.random()*size))
                DBGraph.add_edge((vertex.key,vertex.value),(neighbor.key,neighbor.value))
        return DBGraph


    # gets euclidean distance between 2 vectors
    def get_distance(self,v1,v2):
        return  np.linalg.norm(v1-v2)


    # # gets medoid of a graph
    # def get_medoid(self):
    #     min_distance=10000000000000000000
    #     medoid=None
    #     for vertex in self.DBGraph:
    #         current_total_distance=0
    #         for vertex2 in self.DBGraph:
    #             if(vertex==vertex2):
    #                 continue
    #             dist=self.get_distance(vertex.value,vertex2.value)
    #             current_total_distance+=dist
            
    #         if(current_total_distance<min_distance):
    #             min_distance=current_total_distance
    #             medoid=vertex
    #     print(medoid.key)
    #     return medoid,min_distance
    

    def get_medoid(self):
        vX = list(self.DBGraph.get_vertices())
        Embeddings = [self.DBGraph.verticies[i].value for i in vX]
        # print(Embeddings)                                            # get all vertices
        vMean = np.mean(Embeddings, axis=0)                               # compute centroid
        # print("vMean",vMean)
        i = np.argmin([sum((x - vMean)**2) for x in Embeddings])          # pick a point closest to centroid
        minIndex = self.DBGraph.get_vertex(np.argmin([sum((x - vMean)**2) for x in Embeddings]))
        # print(i)
        return minIndex # pick a point closest to centroid



    #gets arg min distance from any vertex in Anyset to Query and min dist
    def get_min_dist (self,AnyKeysSet,Query):
        min_dist=10000000000000000000
        min_vertex=None
        for vertexKey in AnyKeysSet:
            # print("vertex",vertex)
            # print("Query",Query[:3])
            vertex=self.DBGraph.get_vertex(vertexKey)
            dist=self.get_distance(vertex.value,Query)
            if(dist<min_dist):
                min_dist=dist
                min_vertex=vertex
        return min_vertex,min_dist
    
    def get_min_dist_Key (self,AnyKeysSet,Query):
        arrKey = list(AnyKeysSet)
        arrEmb = [self.DBGraph.verticies[i].value for i in arrKey]
        arrEmb = np.array(arrEmb) - Query
        minDist = np.linalg.norm(arrEmb, axis=1)
        minIndex = np.argmin(minDist)
        return arrKey[minIndex], minDist[minIndex]

        # min_dist=10000000000000000000
        # min_vertex=None
        # for vertexKey in AnyKeysSet:
        #     # print("vertex",vertex)
        #     # print("Query",Query[:3])
        #     vertex=self.DBGraph.get_vertex(vertexKey)
        #     dist=self.get_distance(vertex.value,Query)
        #     if(dist<min_dist):
        #         min_dist=dist
        #         min_vertex=vertex.key
        # return min_vertex,min_dist


    #initially, start is the medoid
    # s is a vertex, Query is a vector
    # k is a number, L is a number
    # TODO: change them to be indices instead of vertices to save ram
    def Greedy_Search(self,start,Query):
        search_List={start.key}
        Visited=set()
        #TODO: make the visited and the possible frontier set of indices instead of vertices to save ram.
        possible_frontier=search_List
        while possible_frontier != set():
            # print('possible_frontier',possible_frontier)
            p_star,_= self.get_min_dist_Key(possible_frontier,Query)
            #
            # print('pstar',p_star)
            # if p_star==None:
            #     # break
            #     print('frontier: ')
            #     for v in possible_frontier:
            #         print(v)
            #     print(possible_frontier==set())
            search_List=search_List.union(self.DBGraph.get_vertex(p_star).neighbors)
            Visited.add(p_star)
            if(len(search_List)>self.L):
                #update search list to retain closes L points to x_q
                search_ListL_L=list(search_List)
                search_ListL_L.sort(key=lambda x: self.get_distance(self.DBGraph.get_vertex(x).value,Query))
                # only maintain L closest points
                search_ListL_L=search_ListL_L[:self.L]
                search_List=set(search_ListL_L)

            possible_frontier=search_List.difference(Visited)
            
        search_ListL_L=list(search_List)
        search_ListL_L.sort(key=lambda x: self.get_distance(self.DBGraph.get_vertex(x).value,Query))
        # only maintain k closest points
        search_ListL_L=search_ListL_L[:self.k]
        search_List=set(search_ListL_L)
        return search_List,Visited

    # # Robust pruning 
    #candidate set is set of integers
    def Robust_Prune(self,point,candidate_set,alpha):
        # print(candidate_set)
        candidate_set=candidate_set.union(point.neighbors)
        candidate_set.difference({point.key}) # changed
        point.neighbors=set()
        # print("candidate_set", candidate_set)
        while candidate_set!=set():
            # print("candidate_set", candidate_set)
            # print("point.value", point.value)
            p_star,_= self.get_min_dist_Key(candidate_set,point.value)
            point.neighbors.add(p_star)
            if(len(point.neighbors)==self.R):
                break
            DummySet=candidate_set.copy()
            for candidatePointKey in candidate_set:
                candidatePoint=self.DBGraph.get_vertex(candidatePointKey)
                if(alpha * self.get_distance(self.DBGraph.get_vertex(p_star).value,candidatePoint.value)<=self.get_distance(candidatePoint.value,point.value)):
                    DummySet.remove(candidatePoint.key)
            candidate_set=DummySet


    def Build_Index(self):

        # R = min(R, len(dataset))

        self.iterationOverGraph(1) #alpha=1
        self.iterationOverGraph(self.alpha) #alpha=2

        
        
    def iterationOverGraph(self,alpha):
        # print('medoid',medoid)
        randIndex = list(self.DBGraph.get_vertices())
        random.shuffle(randIndex)
        # random permutation + sequential graph update
        for n in randIndex:
            node = self.DBGraph.get_vertex(n)
            # print(n)

            (_,V) = self.Greedy_Search(self.medoid, node.value)
            self.Robust_Prune(node, V, alpha)
            neighbors = node.get_neighbors()
            
            for inbKey in neighbors:
                
                # CHECK : The backward edge is always added
                # check here in case we shouldn't add it in all cases ? Might be incorrect? 
                inb=self.DBGraph.get_vertex(inbKey)
                inb.add_neighbor(node.key)
                if len(inb.get_neighbors()) > self.R:
                    # print("inb.get_neighbors()", inb.get_neighbors())
                    self.Robust_Prune(inb, inb.get_neighbors(), alpha)
        

# Convert to binary

In [ ]:
def csv_to_binary(csv_file_path, binary_file_path):
    # Read the CSV data
    data = pd.read_csv(csv_file_path, header=None)
    # print(data)
    # Write the data to a binary file
    with open(binary_file_path, 'wb') as f:
        pickle.dump(data, f)

csv_to_binary('dataset/DB10.csv','dataset/DB10.bin')

# Main cell

In [98]:
R=5
k=10
L=15
alpha=2
path='dataset/DB1K.bin'

myDB= VecDB(R,L,alpha,k,path)
myDB.Build_Index()


# converting to Networkx


In [32]:
G = nx.DiGraph()
G.add_nodes_from(range(2))
# G = nx.path_graph(2)

attrs = {0: {"vec": [1,2,3,4]}}
nx.set_node_attributes(G, attrs)
G.nodes[0]["vec"]

attrs = {1: {"vec": [2,3,4]}}
nx.set_node_attributes(G, attrs)
print(G.nodes())
print(G.nodes[1]["vec"])

G.add_edge(0,1)
G.add_edge(1,0)

G.edges()



[0, 1]
[2, 3, 4]


OutEdgeView([(0, 1), (1, 0)])